 # User Registration and Authentication 
# Objectives:

- Implement user registration with unique usernames and passwords.
- Add login functionality to authenticate users.
# Approach:

- Use Python's sqlite3 to store users' data.
- Store passwords securely using hashing (e.g., bcrypt or hashlib).
# Steps:

- Database Setup:

- Create a table users with columns for username, password_hash, and email.
- User Registration:

- Take input from the user for username, password, and email.
- Hash the password before storing it in the database.
# Login Functionality:

- Verify the username and password by comparing the hashed version of the entered password with the stored hash

In [16]:
import sqlite3
import bcrypt

def create_user_table():
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users
                      (id INTEGER PRIMARY KEY, username TEXT, password_hash TEXT, email TEXT)''')
    conn.commit()
    conn.close()

def register_user(username, password, email):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    password_hash = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    cursor.execute('INSERT INTO users (username, password_hash, email) VALUES (?, ?, ?)', (username, password_hash, email))
    conn.commit()
    conn.close()

def authenticate_user(username, password):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('SELECT password_hash FROM users WHERE username = ?', (username,))
    result = cursor.fetchone()
    conn.close()
    if result and bcrypt.checkpw(password.encode('utf-8'), result[0]):
        return True
    return False


# Income and Expense Tracking 
# Objectives:

- Allow users to add, update, and delete income and expense entries.
- Categorize transactions (e.g., Food, Rent, Salary).
# Approach:

- Use income and expense tables with columns like amount, category, date, type (income/expense).
- Provide functions for CRUD operations: Add, Update, Delete.
# Steps:

- Create income and expenses tables.
- Add methods for adding, editing, and deleting entries.
- Display all entries with filters by type and category.

In [17]:
def create_transactions_table():
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS transactions
                      (id INTEGER PRIMARY KEY, user_id INTEGER, amount REAL, category TEXT, date TEXT, type TEXT)''')
    conn.commit()
    conn.close()

def add_transaction(user_id, amount, category, date, transaction_type):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO transactions (user_id, amount, category, date, type) VALUES (?, ?, ?, ?, ?)', 
                   (user_id, amount, category, date, transaction_type))
    conn.commit()
    conn.close()

def update_transaction(transaction_id, amount, category, date, transaction_type):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''UPDATE transactions 
                      SET amount = ?, category = ?, date = ?, type = ? 
                      WHERE id = ?''', (amount, category, date, transaction_type, transaction_id))
    conn.commit()
    conn.close()

def delete_transaction(transaction_id):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('DELETE FROM transactions WHERE id = ?', (transaction_id,))
    conn.commit()
    conn.close()

def get_transactions(user_id, transaction_type=None):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    if transaction_type:
        cursor.execute('SELECT * FROM transactions WHERE user_id = ? AND type = ?', (user_id, transaction_type))
    else:
        cursor.execute('SELECT * FROM transactions WHERE user_id = ?', (user_id,))
    transactions = cursor.fetchall()
    conn.close()
    return transactions


# Financial Reports 
# Objectives:

- Generate monthly and yearly financial reports.
- Calculate total income, expenses, and savings.
# Approach:

- Create queries to aggregate the sum of income and expenses for the required period (monthly, yearly).
- Calculate savings as the difference between income and expenses.

In [18]:
from datetime import datetime

def get_monthly_report(user_id, month, year):
    start_date = f"{year}-{month}-01"
    end_date = f"{year}-{month}-31"
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''SELECT SUM(amount) FROM transactions 
                      WHERE user_id = ? AND type = 'income' AND date BETWEEN ? AND ?''', 
                   (user_id, start_date, end_date))
    total_income = cursor.fetchone()[0] or 0
    cursor.execute('''SELECT SUM(amount) FROM transactions 
                      WHERE user_id = ? AND type = 'expense' AND date BETWEEN ? AND ?''', 
                   (user_id, start_date, end_date))
    total_expenses = cursor.fetchone()[0] or 0
    savings = total_income - total_expenses
    conn.close()
    return total_income, total_expenses, savings


# Budgeting 
# Objectives:

- Enable users to set monthly budgets for different categories.
- Notify users when they exceed their budget limits.
# Approach:

- Create a table for storing budget limits by category and month.
- Compare the total expenses in each category against the set budget.

In [19]:
def create_budget_table():
    try:
        conn = sqlite3.connect('finance_manager.db')
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS budgets
                          (user_id INTEGER, month INTEGER, year INTEGER, category TEXT, limit REAL)''')
        conn.commit()
        conn.close()
        print("Budgets table created successfully or already exists.")
    except sqlite3.Error as e:
        print(f"Error creating budgets table: {e}")


def set_budget(user_id, month, year, category, budget_limit):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('''REPLACE INTO budgets (user_id, month, year, category, limit) 
                      VALUES (?, ?, ?, ?, ?)''', (user_id, month, year, category, budget_limit))
    conn.commit()
    conn.close()

def check_budget_exceed(user_id, month, year, category):
    conn = sqlite3.connect('finance_manager.db')
    cursor = conn.cursor()
    cursor.execute('SELECT limit FROM budgets WHERE user_id = ? AND month = ? AND year = ? AND category = ?', 
                   (user_id, month, year, category))
    budget_limit = cursor.fetchone()
    if not budget_limit:
        return None
    budget_limit = budget_limit[0]
    cursor.execute('''SELECT SUM(amount) FROM transactions 
                      WHERE user_id = ? AND type = 'expense' AND category = ? 
                      AND date BETWEEN ? AND ?''', 
                   (user_id, category, f"{year}-{month}-01", f"{year}-{month}-31"))
    total_expenses = cursor.fetchone()[0] or 0
    conn.close()
    return total_expenses > budget_limit


# Data Persistence 
# Objectives:

- Store data and transactions in a database.
- Implement functions for backup and restore.
# Approach:

- Implement backup functionality using SQLite's .backup() method.
- Create a restore function to load a backup into the current database.# 

In [20]:
def backup_database():
    conn = sqlite3.connect('finance_manager.db')
    backup_conn = sqlite3.connect('finance_manager_backup.db')
    conn.backup(backup_conn)
    backup_conn.close()
    conn.close()

def restore_database():
    conn = sqlite3.connect('finance_manager.db')
    backup_conn = sqlite3.connect('finance_manager_backup.db')
    backup_conn.backup(conn)
    backup_conn.close()
    conn.close()


# Testing and Documentation 
# Objectives:

- Write unit tests for core functionalities.
- Create a user manual with installation and usage instructions.
# Approach:

- Use unittest to write unit tests for user registration, transaction management, report generation, etc.
- Provide a clear user manual explaining installation, configuration, and basic usage of the app.



In [21]:
import unittest

class TestFinanceManager(unittest.TestCase):
    def test_user_registration(self):
        register_user('test_user', 'password123', 'test@example.com')
        self.assertTrue(authenticate_user('test_user', 'password123'))

    def test_add_transaction(self):
        add_transaction(1, 100, 'Salary', '2024-12-01', 'income')
        transactions = get_transactions(1)
        self.assertEqual(len(transactions), 1)

    def test_monthly_report(self):
        total_income, total_expenses, savings = get_monthly_report(1, 12, 2024)
        self.assertIsInstance(total_income, float)
        self.assertIsInstance(total_expenses, float)
        self.assertIsInstance(savings, float)


In [25]:
def main():
    try:
        # Setup (creating tables if they don't exist)
        create_user_table()
        create_transactions_table()
        create_budget_table()

        # Ask for user action (register or login)
        print("Welcome to the Personal Finance Manager!")
        action = input("Would you like to (r)egister or (l)ogin? (r/l): ").strip().lower()

        if action == 'r':
            # Register a new user
            username = input("Enter a username: ").strip()
            password = input("Enter a password: ").strip()
            email = input("Enter your email: ").strip()
            register_user(username, password, email)
            print(f"User {username} registered successfully!")

        elif action == 'l':
            # Login existing user
            username = input("Enter your username: ").strip()
            password = input("Enter your password: ").strip()
            
            if authenticate_user(username, password):
                print("User authenticated successfully!")
            else:
                print("Authentication failed!")
                return

        else:
            print("Invalid action selected! Exiting.")
            return

        # Once the user is authenticated, allow them to interact with the system
        while True:
            print("\nWhat would you like to do?")
            print("1. Add Transaction")
            print("2. View Transactions")
            print("3. Exit")
            
            choice = input("Enter your choice (1-3): ").strip()

            if choice == '1':
                # Add Transaction
                amount = float(input("Enter the amount: "))
                category = input("Enter the category (e.g., Food, Salary): ").strip()
                date = input("Enter the transaction date (YYYY-MM-DD): ").strip()
                transaction_type = input("Enter transaction type (income/expense): ").strip().lower()

                if transaction_type not in ['income', 'expense']:
                    print("Invalid transaction type! Please enter 'income' or 'expense'.")
                    continue

                # Assuming user_id is 1 for this example
                add_transaction(1, amount, category, date, transaction_type)
                print(f"Transaction of {amount} ({transaction_type}) for {category} added.")

            elif choice == '2':
                # View Transactions
                transactions = get_transactions(1)
                if not transactions:
                    print("No transactions found.")
                else:
                    print("Transactions:")
                    for transaction in transactions:
                        print(transaction)

            elif choice == '3':
                # Exit the application
                print("Exiting the application. Goodbye!")
                break

            else:
                print("Invalid choice! Please select between 1-3.")

    except Exception as e:
        print(f"An error occurred: {e}")
